# Run a behavior cloning behavior

Run a behavior cloning based robot controller on a real robot. 

* Initialize the robot controller and the camera controller, based on the exp
* Load the behavior cloning controller based on the exp, together with the sp
* Initialize a loop which loads an image through the camera, passes it through the sp, the controller and applies the controller output to the robot. 


In [1]:
# verifying pathlib is clean
import pathlib
print(pathlib.__file__)

/usr/lib/python3.10/pathlib.py


In [2]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pprint
import math
import socket
import pathlib
import time
import torch
import numpy as np

from robot.al5d_position_controller import PositionController, RobotPosition

hostname = socket.gethostname()
print(f"Hostname is {hostname}")
if hostname == "raven":
    # the gamepad is only available on some machines, and only on Linux
    available_gamepad = True
    try:
        from robotcontrol.gamepad_controller import GamepadController
    except ModuleNotFoundError:
        print("Approxend module not found, cannot use gamepad")
        available_gamepad = False
    available_robot = True
elif hostname == "szenes.local":
    available_gamepad = False
    available_robot = False
elif hostname == "glassy":
    available_gamepad = False
    available_robot = False
else:
    available_gamepad = False
    available_robot = False
    
from robotcontrol.keyboard_controller import KeyboardController
from robotcontrol.program_controller import ProgramController

# the robot is not always available, this allows us to run the demonstration 
# without the robot


from camera.camera_controller import CameraController
import sensorprocessing.sp_helper as sp_helper
from sensorprocessing.sp_factory import create_sp
import bc_factory 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(f"Using device: {device}")

Hostname is raven
/usr/lib/python3.10/pathlib.py
***ExpRun**: Loading pointer config file:
	/home/al5d/.config/BerryPicker/mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	~/WORK/BerryPicker/cfg/settings.yaml
Using device: cuda


In [3]:
# Setting up a separate directory for generated and computed data
exprun_path, result_path = bc_factory.external_setup("BerryPicker-BC")

Hostname is raven
Path for external experiments: /home/al5d/WORK/_DataExternal/BerryPicker-BC/exprun
Path for external data: /home/al5d/WORK/_DataExternal/BerryPicker-BC/result
***ExpRun**: Experiment config path changed to /home/al5d/WORK/_DataExternal/BerryPicker-BC/exprun
***ExpRun**: Experiment data path changed to /home/al5d/WORK/_DataExternal/BerryPicker-BC/result
***ExpRun**: Experiment demonstration copied to /home/al5d/WORK/_DataExternal/BerryPicker-BC/exprun/demonstration
***ExpRun**: Experiment sensorprocessing_conv_vae copied to /home/al5d/WORK/_DataExternal/BerryPicker-BC/exprun/sensorprocessing_conv_vae
***ExpRun**: Experiment robot_al5d copied to /home/al5d/WORK/_DataExternal/BerryPicker-BC/exprun/robot_al5d
***ExpRun**: Experiment automate copied to /home/al5d/WORK/_DataExternal/BerryPicker-BC/exprun/automate
***ExpRun**: Experiment behavior_cloning copied to /home/al5d/WORK/_DataExternal/BerryPicker-BC/exprun/behavior_cloning
***ExpRun**: Experiment controllers copied 

In [4]:
# load the exp/runs
experiment = "behavior_cloning"
runs = Config().list_runs(experiment)
pprint.pprint(runs)
# this is a specialized run, just for running a robot controller, setting up the camera etc. 
# FIXME: probably I will need to generate these as well from flow...
# FIXME: for the time being I had just edited one in the created directory
# run = "bc_run_00"
run = "runbc_bc_lstm_0001"
exp = Config().get_experiment(experiment, run)

# note that we are loading the exprun for the robot controller even if the robot is not connected
exp_robot_controller = Config().get_experiment(exp["exp_robot_controller"], exp["run_robot_controller"])

if hostname == "glassy":
    exp_camera_controller = Config().get_experiment("controllers", "camera_controller_glassy")
elif hostname == "raven":
    exp_camera_controller = Config().get_experiment("controllers", "camera_controller_raven")
else:
    exp_camera_controller = Config().get_experiment(exp["exp_camera_controller"], exp["run_camera_controller"])

# the behavior cloning exp/run
exp_bc = Config().get_experiment(exp["exp_bc"], exp["run_bc"])
# the sensor processing exp/run
exp_sp = Config().get_experiment(exp_bc["exp_sp"], exp_bc["run_sp"])

['bc_run_0001',
 'bc_verify_mlp_00',
 'runbc_bc_lstm_0001',
 'bc_verify_lstm_mdn_00',
 'mdn_00',
 'bc_mlp_00',
 'bc_lstm_00',
 'bc_lstm_mdn_00',
 'runbc_bc_mlp_0001',
 'bc_mlp_0001',
 'bc_lstm_resid_00',
 'mdn_for_bc_00',
 'bc_lstm_0001',
 'bc_verify_lstm_resid_00',
 'bc_lstm_residual_0001',
 'bc_lstm_mdn_0001',
 'bc_compare_00',
 '_defaults_behavior_cloning',
 'runbc_bc_lstm_residual_0001',
 'bc_0001',
 'bc_run_00',
 'runbc_bc_lstm_mdn_0001',
 'bc_verify_lstm_00']
***ExpRun**: Configuration for exp/run: behavior_cloning/runbc_bc_lstm_0001 successfully loaded
***ExpRun**: Configuration for exp/run: robot_al5d/position_controller_00 successfully loaded
***ExpRun**: Experiment default config /home/al5d/WORK/_DataExternal/BerryPicker-BC/exprun/controllers/_defaults_controllers.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: controllers/camera_controller_raven successfully loaded
***ExpRun**: Configuration for exp/run: behavior_cloning/bc_lstm_0001 successfully loaded
***ExpRun

In [5]:
# starting the robot controller
if available_robot:
    robot_controller = PositionController(exp_robot_controller)
else:
    robot_controller = None
# starting the camera controller
camera_controller = CameraController(exp_camera_controller)
camera_controller.visualize = True

# starting the sensor processing
sp = create_sp(exp_sp, device)

# load the behavior cloning controller
model, _, _ = bc_factory.create_bc_model(exp_bc, exp_sp, device)
model_path = pathlib.Path(exp_bc.data_dir(), exp_bc["controller_file"])
model.load_state_dict(torch.load(model_path))   

***ExpRun**: Configuration for exp/run: robot_al5d/pulse_controller_00 successfully loaded
***ExpRun**: Configuration for exp/run: robot_al5d/angle_controller_00 successfully loaded
{'height': 5.0, 'distance': 5.0, 'heading': 0.0, 'wrist_angle': -45.0, 'wrist_rotation': 75.0, 'gripper': 100}
cap0 works


<All keys matched successfully>

### Running the robot

Run the robot controller in a loop, with a human in the loop test at every move.

In [6]:
def safe_robot_control(a, robot_controller, exp_robot_controller):
    """Moves the robot to the position, with various safety controler"""
    pos_target = RobotPosition.from_normalized_vector(exp_robot_controller,a)
    print(f"Target position: {pos_target}", flush=True)
    if not robot_controller:
        return "0"
    # we assume that we have the robot
    # ask the user whether we indeed want to continue moving?
    pos_current = robot_controller.get_position()
    distance = pos_current.empirical_distance(exp_robot_controller, pos_target)
    query = f"Next position: {pos_target} at distance={distance} Proceed with the move? (A/Y/N/Q)"
    ans = input(query)

    # create an artificial target, changing the heading, just in case

    if ans.lower() == "y": # move towards the target, a safe amound
        pos_target_safe = RobotPosition(exp_robot_controller, pos_current.values)

        diff = (pos_target["height"] - pos_current["height"])
        diff = min(0.1, max(diff, -0.1))
        pos_target_safe["height"] += diff

        diff = (pos_target["heading"] - pos_current["heading"])
        diff = min(2, max(diff, -2))
        pos_target_safe["heading"] += diff


        robot_controller.move(pos_target_safe)
        ""
        return "y"
    elif ans.lower() == "a": # make an artificial move, just to see that we are controlling the robot
        pos_target_artificial = RobotPosition(exp_robot_controller, pos_current.values)
        pos_target_artificial["heading"] = pos_target_artificial["heading"] + 3
        robot_controller.move(pos_target_artificial)
    elif ans.lower() == "q":
        return "q"


In [7]:
def run_robot(model, robot_controller, exp_robot_controller, camera_controller, controller_interval = 0.1):
    """Robot running loop. Loads the image from the camera, runs the model, and executes the recommended code."""
    # last_interval = 0
    controller_interval = 0.1

    transform = sp_helper.get_transform_to_sp(exp_sp)
    # the backlog of the last sequence length steps
    z_seq = []

    while True:
        start_time = time.time() 
        # Let us assume that the first image here is the one
        toexit = camera_controller.update()
        if toexit:
            break
        # FIXME: this should be made different... based on the run
        # image = camera_controller.images[exp["control_camera"]]
        image = camera_controller.images["dev0"]
        # FIXME: this is totally won't work like this
        # there should be a notebook with the input processing???
        sensor_readings = sp_helper.load_capture_to_tensor(image, transform, device)
        z = sp.process(sensor_readings[0])
        print(z)
        if exp_bc["sequence_length"]: # no sequence processsing
            z_seq.append(z)
            if len(z_seq) > exp_bc["sequence_length"]:
                z_seq.pop(0)
            if len(z_seq) < exp_bc["sequence_length"]:
                ## we don't have enough in the queue, wait.
                print("Not acting, waiting on the queue", flush=True)
                z_tensor = None
                pass
            else:
                # FIXME: this should be applied online, not step by step
                z_seq_np = np.array(z_seq)
                z_tensor = torch.tensor(z_seq_np).unsqueeze(0).to(device)
        else:
            z_tensor = torch.tensor(z).unsqueeze(0).to(device)
        if z_tensor == None:
            continue
        if not model.stochastic:                
            a_pred = model(z_tensor)
        else:
            a_pred = model.forward_and_sample(z_tensor)

        retval = safe_robot_control(a_pred[0], robot_controller, exp_robot_controller)
        if retval == "q":
            break

        # ensuring that the execution happens 
        end_time = time.time() 
        execution_time = end_time - start_time 
        # last_interval = execution_time
        time_to_sleep = max(0.0, controller_interval - execution_time) 
        time.sleep(time_to_sleep) 

In [8]:
run_robot(model = model, robot_controller = robot_controller, exp_robot_controller = exp_robot_controller, camera_controller = camera_controller)

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/al5d/WORK/BerryPicker/vm/berrypickervenv/lib/python3.10/site-packages/cv2/qt/plugins"


[ 0.17713988  0.13305105 -0.01849388  0.08952694  0.01289851  0.08536877
 -0.09917244 -0.02870453  0.03386152  0.15901197  0.06612907 -0.11111819
  0.01641092 -0.22302175 -0.13358793 -0.01670615  0.0760164  -0.14985749
  0.14852512 -0.08489003 -0.13735402  0.16744591 -0.18217313  0.10870291
 -0.18203595  0.16388898 -0.00820963 -0.20891388 -0.13636874  0.05964046
  0.24270576  0.15556553 -0.43851638  0.01617312  0.12970485 -0.09932116
 -0.08084598 -0.08681266 -0.07138832  0.06747492  0.04079825  0.050076
  0.06979422  0.25977507  0.04646387 -0.15831552 -0.12934403 -0.03330291
  0.09750488  0.06567908 -0.12994738 -0.00715035  0.08988436  0.10941637
  0.01666897  0.10812019 -0.03600585 -0.01678695 -0.23028831 -0.06430095
  0.07596838 -0.00260716  0.04304826 -0.07720375  0.12688059 -0.08443097
  0.16774334 -0.12239397  0.1233371  -0.02641733  0.11599474  0.05480568
 -0.2290445   0.00301888 -0.13940884  0.09571559  0.0007787   0.25698105
  0.0037155   0.03860903 -0.04934812  0.1373947  -0.0

In [9]:
# shutdown
camera_controller.stop()
if robot_controller:
    robot_controller.stop_robot()